In [2]:
from py2neo import Graph,Node,Relationship
import config


In [3]:
graph = Graph('http://localhost:7474', username='neo4j', password=config['password'])


In [63]:
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
print(a, b, r)

(alice:Person {name:"Alice"}) (bob:Person {name:"Bob"}) (alice)-[:KNOWS]->(bob)


In [64]:
bb = Node(label='Person', name='Bobb')
print(bb)

(bobb {label:"Person",name:"Bobb"})


In [65]:
#以上代码必须在创建了node与relationship下进行添加dict形式才有效
a['age'] = 20
b['age'] = 21
r['time'] = '2017/08/31'
print(a, b, r)

(alice:Person {age:20,name:"Alice"}) (bob:Person {age:21,name:"Bob"}) (alice)-[:KNOWS {time:"2017/08/31"}]->(bob)


In [66]:
#以通过 setdefault() 方法赋值默认属性
a.setdefault('location', '北京')
print(a)

(alice:Person {age:20,location:"北京",name:"Alice"})


In [67]:
# 果赋值了 location 属性，则它会覆盖默认属性
a['location'] = '上海'
a.setdefault('location', '北京')
print(a)

(alice:Person {age:20,location:"上海",name:"Alice"})


In [68]:
#使用 update() 方法对属性批量更新
data = {
'name': 'Amy',
'age': 21
}
a.update(data)
#a就是变量名称
print(a)

(alice:Person {age:21,location:"上海",name:"Amy"})


In [69]:
# Subgraph子图
# 子图，是 Node 和 Relationship 的集合，可通过关系运算符
from py2neo import Node, Relationship

a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
s = a | b | r
print(s)

({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})}, {(alice)-[:KNOWS]->(bob)})


In [71]:

#通过 nodes和 relationships属性获取所有的 Node 和 Relationship
print(s.nodes())
print(s.relationships())

frozenset({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})})
frozenset({(alice)-[:KNOWS]->(bob)})


In [13]:
#还可以利用 & 取 Subgraph 的交集
s1 = a | b | r
s2 = a | b
print(s1 & s2)


({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})}, {})


In [14]:
#查询基本属性
s = a | b | r
print('节点属性名称：',s.keys())
print(s.types())

print(s.labels())
print(s.nodes())
print(s.relationships())


frozenset({'name'})
frozenset({'KNOWS'})
frozenset({'Person'})
frozenset({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})})
frozenset({(alice)-[:KNOWS]->(bob)})


In [17]:
##############有问题
#用 order() 或 size() 方法来获取 Subgraph 的 Node 数量和 Relationship 数量
from py2neo import Node, Relationship, size, order
s = a | b | r
print('node 数量',order(s))
print('relationship数量',size(s))

node 数量 2
relationship数量 1


In [18]:
from py2neo import Node, Relationship
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
c = Node('Person', name='Mike')
ab = Relationship(a, "KNOWS", b)
ac = Relationship(a, "KNOWS", c)


In [19]:

w = ab + Relationship(b, "LIKES", c) + ac
print(w)

(alice)-[:KNOWS]->(bob)-[:LIKES]->(mike)<-[:KNOWS]-(alice)


In [20]:
#可以调用 walk() 方法实现遍历
from py2neo import walk

for item in walk(w):
    print(item)

(alice:Person {name:"Alice"})
(alice)-[:KNOWS]->(bob)
(bob:Person {name:"Bob"})
(bob)-[:LIKES]->(mike)
(mike:Person {name:"Mike"})
(alice)-[:KNOWS]->(mike)
(alice:Person {name:"Alice"})


In [21]:
print(w.start_node())
print(w.end_node())
print(w.nodes())
print(w.relationships())

(alice:Person {name:"Alice"})
(alice:Person {name:"Alice"})
((alice:Person {name:"Alice"}), (bob:Person {name:"Bob"}), (mike:Person {name:"Mike"}), (alice:Person {name:"Alice"}))
((alice)-[:KNOWS]->(bob), (bob)-[:LIKES]->(mike), (alice)-[:KNOWS]->(mike))


In [24]:
#利用 create() 方法传入 Subgraph 对象来将关系图添加到数据库中
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
s = a | b | r
graph = Graph('http://localhost:7474', username='neo4j', password='LW199112262315.')
graph.create(s)

In [25]:
print(graph)

<Graph uri='http://localhost:7474/db/data/'>


In [26]:
#有问题利用 data() 方法来获取查询结果：
data = graph.data('MATCH (p:Person) return p')
print(data)

[{'p': (ed2e008:Person {born:1964,name:"Keanu Reeves"})}, {'p': (f0b777a:Person {born:1967,name:"Carrie-Anne Moss"})}, {'p': (a11a43a:Person {born:1961,name:"Laurence Fishburne"})}, {'p': (a654db9:Person {born:1960,name:"Hugo Weaving"})}, {'p': (a1c2554:Person {born:1967,name:"Lilly Wachowski"})}, {'p': (b1d0ba2:Person {born:1965,name:"Lana Wachowski"})}, {'p': (af2d3d9:Person {born:1952,name:"Joel Silver"})}, {'p': (b816746:Person {born:1978,name:"Emil Eifrem"})}, {'p': (eef64c4:Person {born:1975,name:"Charlize Theron"})}, {'p': (a39c019:Person {born:1940,name:"Al Pacino"})}, {'p': (f16df50:Person {born:1944,name:"Taylor Hackford"})}, {'p': (e60929f:Person {born:1962,name:"Tom Cruise"})}, {'p': (c463103:Person {born:1937,name:"Jack Nicholson"})}, {'p': (c2e8800:Person {born:1962,name:"Demi Moore"})}, {'p': (d6e4a7e:Person {born:1958,name:"Kevin Bacon"})}, {'p': (da9453e:Person {born:1966,name:"Kiefer Sutherland"})}, {'p': (c490c4e:Person {born:1971,name:"Noah Wyle"})}, {'p': (a157034:

In [27]:
#输出结果还可以直接转化为 DataFrame 对象
from pandas import DataFrame
df = DataFrame(data)
print(df.head())

                                              p
0        {'name': 'Keanu Reeves', 'born': 1964}
1    {'name': 'Carrie-Anne Moss', 'born': 1967}
2  {'name': 'Laurence Fishburne', 'born': 1961}
3        {'name': 'Hugo Weaving', 'born': 1960}
4     {'name': 'Lilly Wachowski', 'born': 1967}


In [28]:
#使用 find_one() 或 find() 方法进行 Node 的查找，
# 可以利用 match() 或 match_one() 方法对 Relationship 进行查找

#Graph.find_one is deprecated, use NodeSelector instead
node = graph.find_one(label='Person')
print(node)

relationship = graph.match_one(rel_type='KNOWS')
print(relationship)

F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """


(ed2e008:Person {born:1964,name:"Keanu Reeves"})


(ad0ba45)-[:KNOWS]->(e821801)


In [29]:
# 如果想要更新 Node 的某个属性可以使用 push() 方法，例如：
node = graph.find_one(label='Person')
node['age'] = 21
graph.push(node)
print(graph.find_one(label='Person'))



F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  


(ed2e008:Person {age:21,born:1964,name:"Keanu Reeves"})


F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """


In [35]:
node = graph.find_one(label='Person')
relationship = graph.match_one(rel_type='KNOWS')

print(node)
print(relationship)


#在删除 Node 时必须先删除其对应的 Relationship，否则无法删除 Node。
graph.delete(relationship)
graph.delete(node)

(ed2e008:Person {age:21,born:1964,name:"Keanu Reeves"})
(alice)-[:KNOWS]->(bob)


F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """Entry point for launching an IPython kernel.


In [8]:
data2=graph.data('MATCH (p:Person {age:22}) RETURN p LIMIT 5')
print(list(data2))

[]


In [4]:
#通过 run() 方法直接执行 CQL 语句
data = graph.run('MATCH (p:Person) RETURN p LIMIT 5')
print(list(data))

[('p': (e0dc9ba:Person {born:1964,name:"Keanu Reeves"})), ('p': (b088b01:Person {born:1967,name:"Carrie-Anne Moss"})), ('p': (d610ea0:Person {born:1961,name:"Laurence Fishburne"})), ('p': (c3e0bd8:Person {born:1960,name:"Hugo Weaving"})), ('p': (e4a6a08:Person {born:1967,name:"Lilly Wachowski"}))]


In [9]:
#NodeSelector可多个节点查询，graph不行
from py2neo import NodeSelector
selector=NodeSelector(graph)
persons = selector.select('Person', age=21)
print('first\n',list(persons))


first
 []


In [47]:
#加上where用复杂查询
persons = selector.select('Person').where('_.name =~ "A.*"')
print('second\n',list(persons))

second
 [(ae3f0ca:Person {born:1961,name:"Aaron Sorkin"}), (a39c019:Person {born:1940,name:"Al Pacino"}), (alice:Person {name:"Alice"}), (f911680:Person {name:"Angela Scope"}), (a6cbda6:Person {born:1960,name:"Annabella Sciorra"}), (dffc1be:Person {born:1962,name:"Anthony Edwards"}), (b880076:Person {born:1976,name:"Audrey Tautou"})]


In [48]:
# 也可以使用 order_by() 进行排序：
persons = selector.select('Person').order_by('_.age')
print(list(persons))

[(ed2e008:Person {age:21,born:1964,name:"Keanu Reeves"}), (f0b777a:Person {born:1967,name:"Carrie-Anne Moss"}), (a11a43a:Person {born:1961,name:"Laurence Fishburne"}), (a654db9:Person {born:1960,name:"Hugo Weaving"}), (a1c2554:Person {born:1967,name:"Lilly Wachowski"}), (b1d0ba2:Person {born:1965,name:"Lana Wachowski"}), (af2d3d9:Person {born:1952,name:"Joel Silver"}), (b816746:Person {born:1978,name:"Emil Eifrem"}), (eef64c4:Person {born:1975,name:"Charlize Theron"}), (a39c019:Person {born:1940,name:"Al Pacino"}), (f16df50:Person {born:1944,name:"Taylor Hackford"}), (e60929f:Person {born:1962,name:"Tom Cruise"}), (c463103:Person {born:1937,name:"Jack Nicholson"}), (c2e8800:Person {born:1962,name:"Demi Moore"}), (d6e4a7e:Person {born:1958,name:"Kevin Bacon"}), (da9453e:Person {born:1966,name:"Kiefer Sutherland"}), (c490c4e:Person {born:1971,name:"Noah Wyle"}), (a157034:Person {born:1968,name:"Cuba Gooding Jr."}), (ed57aaa:Person {born:1957,name:"Kevin Pollak"}), (ecd472b:Person {born:1

In [49]:
person = selector.select('Person').where('_.name =~ "A.*"').first()
print(person)

(ae3f0ca:Person {born:1961,name:"Aaron Sorkin"})


In [51]:
#OGM 类似于 ORM，意为 Object Graph Mapping，这样可以实现一个对象和 Node 的关联
from py2neo.ogm import GraphObject, Property, RelatedTo, RelatedFrom


class Movie(GraphObject):
    __primarykey__ = 'title'
    
    title = Property()
    released = Property()
    actors = RelatedFrom('Person', 'ACTED_IN')
    directors = RelatedFrom('Person', 'DIRECTED')
    producers = RelatedFrom('Person', 'PRODUCED')

class Person(GraphObject):
    __primarykey__ = 'name'
    
    name = Property()
    born = Property()
    acted_in = RelatedTo('Movie')
    directed = RelatedTo('Movie')
    produced = RelatedTo('Movie')

In [52]:
#以用它来结合 Graph 查询
from py2neo.ogm import GraphObject, Property
class Person(GraphObject):
    __primarykey__ = 'name'

    name = Property()
    age = Property()
    location = Property()

person = Person.select(graph).where(age=21).first()
print(person)
print(person.name)
print(person.age)


<Person name='Keanu Reeves'>
Keanu Reeves
21


In [53]:
#成功实现了对象和 Node 的映射。
# 我们可以用它动态改变 Node 的属性，例如修改某个 Node 的 age 属性
person = Person.select(graph).where(age=21).first()
print(person.__ogm__.node)
person.age = 22
print(person.__ogm__.node)
graph.push(person)

(ed2e008:Person {age:21,born:1964,name:"Keanu Reeves"})
(ed2e008:Person {age:22,born:1964,name:"Keanu Reeves"})


In [57]:
from py2neo.ogm import GraphObject, Property, RelatedTo



class Person(GraphObject):
    __primarykey__ = 'name'

    name = Property()
    age = Property()
    location = Property()
    knows = RelatedTo('Person', 'KNOWS')

person = Person.select(graph).where(age=22).first()
print(list(person.knows))
new_person = Person()
new_person.name = 'Durant'
new_person.age = 28
person.knows.add(new_person)
print(list(person.knows))

#这样我们就完成了 Node 和 Relationship 的添加，同时由于设置了 primarykey 为 name，
# 所以不会重复添加


[]
[<Person name='Durant'>]


In [59]:
# 也可以通过 remove() 方法移除某个关联 Node
person = Person.select(graph).where(name='Bob').first()
target = Person.select(graph).where(name='Durant').first()
person.knows.remove(target)
graph.push(person)
graph.delete(target)

#这里 target 是 name 为 Durant 的 Node，代码运行完毕后即可删除关联 Relationship 和删除 Node。
# 以上便是 OGM 的用法，查询修改非常方便，推荐使用此方法进行 Node 和 Relationship 的修改



TypeError: No method defined to delete object None

In [72]:
#案例

from py2neo import Graph, Node, Relationship

# 连接neo4j数据库

# 创建结点：label结点，方便以后的结点查找操作
temp_node1 = Node(lable="Person",name="node1")
temp_node2 = Node(lable="Person",name="node2")
graph.create(temp_node1)
graph.create(temp_node2)
# 建立关系
node_1_call_node_2 = Relationship(temp_node1,'CALL',temp_node2)
node_1_call_node_2['count'] = 1
node_2_call_node_1 = Relationship(temp_node2,'CALL',temp_node1)
graph.create(node_2_call_node_1)
graph.create(node_1_call_node_2)
# 更新关系或节点的属性 push提交
node_1_call_node_2['count']+=1
graph.push(node_1_call_node_2)




In [76]:
# 
find_code = graph.find_one(
label="明教",
property_key="name",
property_value="张无忌"
)
print(find_code['name'])

# find方法已被弃用：通过属性值来查找所有节点和关系find替换为：NodeSelector
find = NodeSelector(graph).select('明教')
for f in find:
    print(f['name'])

[]


F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Graph.find is deprecated, use NodeSelector instead
  


In [85]:
all_nodes=graph.run('MATCH (n) RETURN n')

In [87]:
print('all_nodes',all_nodes)

all_nodes <py2neo.database.Cursor object at 0x0000022D1E0D6F98>


In [90]:
cursor_graph=graph.run('''MATCH (n)-->(m)
RETURN n, m;''')

In [94]:
print(dir(cursor_graph))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_current', '_source', 'close', 'current', 'data', 'dump', 'evaluate', 'forward', 'keys', 'next', 'stats']


In [97]:
print(cursor_graph.keys())

['n', 'm']


AttributeError: 'Cursor' object has no attribute 'nodes'